## Predicting Early Readmission in Diabetic Patients

### Project Overview
This project analyzes ten years (1999–2008) of clinical care data from 130 US hospitals and integrated delivery networks. The dataset focuses on diabetic patients who received inpatient care—including lab work, medications, and short hospital stays (up to 14 days). The goal is to predict whether a patient will be readmitted within 30 days of discharge.

#### Why This Matters
Think of it like this: If someone with diabetes goes to the hospital and is sent home too early or without the right care, they might get worse and have to come back. That’s bad for their health and expensive for everyone involved.

This research helps us figure out how to prevent that from happening in the first place.

Even though we know how to improve outcomes for diabetic patients, many still don’t receive proper care during their hospital stay. This project aims to highlight patterns in the data that can be used to flag high-risk patients before discharge—so hospitals can intervene early and reduce unnecessary readmissions.



### Objectives
1. Explore and clean real-world hospital data

2. Identify key features influencing early readmission

3. Train and evaluate machine learning models to predict readmission risk

4. Provide insights to help improve diabetic patient care and hospital efficiency



### Dataset
Source: UCI Machine Learning Repository - Diabetes 130-US hospitals dataset

Size: 100,000+ patient records

Features include demographics, lab tests, medications, diagnosis codes, length of stay, and discharge outcomes

The final model can help hospitals:

Identify patients at risk of early readmission

Improve discharge planning and follow-up care

Reduce healthcare costs and improve patient outcomes



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option('display.max_columns', None)
sns.set(style='whitegrid')

In [2]:
diabetes = pd.read_csv('diabetic_data.csv')
diabetes.shape

(101766, 50)

Let us check out the state of our dataset.

In [3]:
diabetes.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [4]:
diabetes.isnull().sum()

encounter_id                0
patient_nbr                 0
race                        0
gender                      0
age                         0
weight                      0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
payer_code                  0
medical_specialty           0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
diag_2                      0
diag_3                      0
number_diagnoses            0
max_glu_serum               0
A1Cresult                   0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazo

The data could be having "?" as a place filler for null values.

In [5]:
diabetes.apply(lambda col: (col == '?').sum()).sort_values(ascending=False)


weight                      98569
medical_specialty           49949
payer_code                  40256
race                         2273
diag_3                       1423
diag_2                        358
diag_1                         21
num_procedures                  0
max_glu_serum                   0
number_diagnoses                0
number_inpatient                0
number_emergency                0
number_outpatient               0
num_medications                 0
readmitted                      0
num_lab_procedures              0
diabetesMed                     0
time_in_hospital                0
admission_source_id             0
discharge_disposition_id        0
admission_type_id               0
age                             0
gender                          0
patient_nbr                     0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
change                          0
metformin-piog

In [6]:
diabetes.replace('?', np.nan, inplace=True)

Let us check categorical columns with many unique values

In [7]:
for col in diabetes.columns:
    if diabetes[col].dtype == 'object':
        unique_vals = diabetes[col].nunique()
        if unique_vals < 20:
            print(f"{col}: {diabetes[col].unique()}")

race: ['Caucasian' 'AfricanAmerican' nan 'Other' 'Asian' 'Hispanic']
gender: ['Female' 'Male' 'Unknown/Invalid']
age: ['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
weight: [nan '[75-100)' '[50-75)' '[0-25)' '[100-125)' '[25-50)' '[125-150)'
 '[175-200)' '[150-175)' '>200']
payer_code: [nan 'MC' 'MD' 'HM' 'UN' 'BC' 'SP' 'CP' 'SI' 'DM' 'CM' 'CH' 'PO' 'WC' 'OT'
 'OG' 'MP' 'FR']
max_glu_serum: ['None' '>300' 'Norm' '>200']
A1Cresult: ['None' '>7' '>8' 'Norm']
metformin: ['No' 'Steady' 'Up' 'Down']
repaglinide: ['No' 'Up' 'Steady' 'Down']
nateglinide: ['No' 'Steady' 'Down' 'Up']
chlorpropamide: ['No' 'Steady' 'Down' 'Up']
glimepiride: ['No' 'Steady' 'Down' 'Up']
acetohexamide: ['No' 'Steady']
glipizide: ['No' 'Steady' 'Up' 'Down']
glyburide: ['No' 'Steady' 'Up' 'Down']
tolbutamide: ['No' 'Steady']
pioglitazone: ['No' 'Steady' 'Up' 'Down']
rosiglitazone: ['No' 'Steady' 'Up' 'Down']
acarbose: ['No' 'Steady' 'Up' 'Down']
miglitol: ['No' 

Since we will be looking at the readmitted coulumn, we can simplify it so that if Readmitted ="<30" we have a 1 if else we have a 0.(Yes or No)

In [8]:
# Binary target: 1 if readmitted within 30 days, else 0
diabetes['readmit_30'] = diabetes['readmitted'].apply(lambda x: 1 if x == '<30' else 0)

In [9]:
diabetes.drop(['weight', 'payer_code', 'encounter_id', 'patient_nbr'], axis=1, inplace=True)

In [10]:
diabetes.duplicated().sum()

0

In [11]:
diabetes.isnull().sum()

race                         2273
gender                          0
age                             0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide                       0
tolbutamide                     0
pioglitazone  

In [13]:
diabetes['medical_specialty'].fillna('Unknown', inplace=True)
diabetes['race'].fillna('Unknown', inplace=True)

In [14]:
diabetes.isnull().sum()

race                           0
gender                         0
age                            0
admission_type_id              0
discharge_disposition_id       0
admission_source_id            0
time_in_hospital               0
medical_specialty              0
num_lab_procedures             0
num_procedures                 0
num_medications                0
number_outpatient              0
number_emergency               0
number_inpatient               0
diag_1                        21
diag_2                       358
diag_3                      1423
number_diagnoses               0
max_glu_serum                  0
A1Cresult                      0
metformin                      0
repaglinide                    0
nateglinide                    0
chlorpropamide                 0
glimepiride                    0
acetohexamide                  0
glipizide                      0
glyburide                      0
tolbutamide                    0
pioglitazone                   0
rosiglitaz

In [17]:
diabetes["diag_2"].dtype

dtype('O')

In [18]:
diabetes['diag_1'].fillna('Missing', inplace=True)
diabetes['diag_2'].fillna('Missing', inplace=True)
diabetes['diag_3'].fillna('Missing', inplace=True)